In [1]:
%load_ext lab_black

In [160]:
import json
import gzip
import pandas as pd
import glob
import tempfile
import uuid
import os
from pprint import pprint
from multiprocessing import Pool
from pathlib import Path
from sadie.airr import Airr, AirrTable
from gspread_pandas import Spread, Client
from sadie.airr import AirrTable
from sadie.airr.constants import IGBLAST_AIRR

spread = Spread("Sadie", create_spread=True, create_sheet=True)
pd.set_option("display.max_rows", 45)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 100000)

In [170]:
from importlib import reload
from sadie.airr import constants

reload(constants)

<module 'sadie.airr.constants' from '/Users/jordanwillis/repos/personal/sadie/src/sadie/airr/constants.py'>

## Read OAS

In [5]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample.bz2",
    index_col=0,
).reset_index()

/Users/jordanwillis/miniconda3/envs/sadie/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (27,43,44,45,46,71,105) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
sub_sample_file = "../../tests/integration/airr/fixtures/OAS_subsample.fasta"
with open("../../tests/integration/airr/fixtures/OAS_subsample.fasta", "w") as f:
    for index, (i, k, j) in enumerate(
        zip(
            df_bz2["OAS_dataunit"].str.split(".csv.gz").str.get(0),
            df_bz2["OAS_dataindex"],
            df_bz2["sequence"],
        )
    ):
        f.write(f">{index}_{i}_{k}\n{j}\n")

## Sadie Annotate

In [210]:
from distutils.version import StrictVersion


def fillna(df, fill_value=""):
    """
    Replace null values with `fill_value`.

    Also replaces in categorical columns.
    """
    for col in df.dtypes[df.dtypes == "category"].index:
        if fill_value not in df[col].cat.categories:
            df[col].cat.add_categories([fill_value], inplace=True)
    # Known bug https://github.com/pandas-dev/pandas/issues/25472
    if StrictVersion(pd.__version__) >= StrictVersion("1.0"):
        for col in df.dtypes[
            df.dtypes.apply(lambda x: x in ["float64", "Int16", "Int64"])
        ].index:
            df[col] = df[col].astype("float")
    return df.fillna(fill_value)

In [ ]:
airr_api = Airr(species="human", database="imgt", functional="all")
sadie_airr = airr_api.run_file(sub_sample_file)

In [21]:
# spreadable_df = fillna(sadie_airr.table.head(100))
spread.df_to_sheet(
    spreadable_df.set_index("sequence_id"),
    sheet="sadie",
    replace=True,
    freeze_headers=True,
    freeze_index=True,
)

## IMGT HI-Vqeust "AIRR" format

In [162]:
imgt_airr = "../../tests/integration/airr/fixtures/OAS_airr_from_imgtvquest.tsv"
imgt_df = pd.read_csv(imgt_airr, delimiter="\t")

/Users/jordanwillis/miniconda3/envs/sadie/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10,21,22,44,87,88,95,96) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
spread.df_to_sheet(
    imgt_df.head(100).set_index("sequence_id"),
    sheet="imgt",
    replace=True,
    freeze_headers=True,
    freeze_index=True,
)

In [164]:
top_100_imgt = imgt_df.head(100).copy()
top_100_sadie = sadie_airr.table.head(100).copy()
compariables = []

## Clean UP IMGT

In [184]:
keys = list(constants.IGBLAST_AIRR.keys())
keys.remove("v_frameshift")

In [256]:
def map_bool(df, col):
    return df.loc[:, col].map(
        {"T": True, "F": False, True: True, False: False, nan: False},
        na_action="ignore",
    )


top_100_imgt_airr_only = top_100_imgt[keys].copy()
bool_cols = ["vj_in_frame", "productive", "rev_comp", "complete_vdj", "stop_codon"]
for col in bool_cols:
    top_100_imgt_airr_only[col] = map_bool(top_100_imgt_airr_only, col)


upper_columns = [
    "fwr1",
    "cdr1",
    "fwr2",
    "cdr2",
    "fwr3",
    "cdr3",
    "fwr4",
    "germline_alignment",
    "germline_alignment_aa",
    "sequence_alignment",
    "sequence_alignment_aa",
    "v_germline_alignment",
    "v_germline_alignment_aa",
    "v_sequence_alignment",
    "v_sequence_alignment_aa",
    "d_germline_alignment",
    "d_germline_alignment_aa",
    "d_sequence_alignment",
    "d_sequence_alignment_aa",
    "j_germline_alignment",
    "j_germline_alignment_aa",
    "j_sequence_alignment",
    "j_sequence_alignment_aa",
    "np1",
    "np2",
    "junction",
    "sequence",
]
for col in upper_columns:
    top_100_imgt_airr_only[col] = (
        top_100_imgt_airr_only[col].str.upper().str.replace(".", "")
    )

ignore = [
    "v_cigar",
    "d_cigar",
    "j_cigar",
    "v_call",
    "d_call",
    "j_call",
    "v_score",
    "d_score",
    "j_score",
    "v_support",
    "d_support",
    "j_support",
    "v_germline_start",
    "v_alignment_end",
    "v_germline_end",
]

starts_and_ends = [
    "cdr1_end",
    "cdr1_start",
    "cdr2_end",
    "cdr2_start",
    "cdr3_end",
    "cdr3_start",
    "d_alignment_end",
    "d_alignment_start",
    "d_germline_end",
    "d_germline_start",
    "d_sequence_end",
    "d_sequence_start",
    "fwr1_end",
    "fwr1_start",
    "fwr2_end",
    "fwr2_start",
    "fwr4_end",
    "fwr4_start",
    "j_alignment_end",
    "j_alignment_start",
    "j_germline_end",
    "j_germline_start",
    "j_sequence_end",
    "j_sequence_start",
]

top_100_imgt_airr_only["v_call_top"] = (
    top_100_imgt_airr_only["v_call"].str.split(",").str.get(0).str.split().str.get(1)
)
top_100_imgt_airr_only["d_call_top"] = (
    top_100_imgt_airr_only["d_call"]
    .str.split(",")
    .str.get(0)
    .str.split("_")
    .str.get(-1)
)
top_100_imgt_airr_only["j_call_top"] = (
    top_100_imgt_airr_only["j_call"].str.split(",").str.get(0).str.split().str.get(1)
)
top_100_imgt_airr_only = top_100_imgt_airr_only.drop(ignore, axis=1)
top_100_imgt_airr_only.loc[:, starts_and_ends] = top_100_imgt_airr_only[
    starts_and_ends
].astype("Int64")
top_100_imgt_airr_only = top_100_imgt_airr_only.set_index("sequence_id")

<ipython-input-256-8915dee3e74c>:45: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  top_100_imgt_airr_only[col].str.upper().str.replace(".", "")


## Clean up sadie

In [257]:
top_100_sadie_airr_only = top_100_sadie[keys]
top_100_sadie_airr_only["v_call_top"] = (
    top_100_sadie_airr_only["v_call"].str.split(",").str.get(0)
)
top_100_sadie_airr_only["d_call_top"] = (
    top_100_sadie_airr_only["d_call"].str.split(",").str.get(0)
)
top_100_sadie_airr_only["j_call_top"] = (
    top_100_sadie_airr_only["j_call"].str.split(",").str.get(0)
)
top_100_sadie_airr_only = top_100_sadie_airr_only.drop(ignore, axis=1).set_index(
    "sequence_id"
)
top_100_sadie_airr_only.loc[:, starts_and_ends] = top_100_sadie_airr_only[
    starts_and_ends
].astype("Int64")

<ipython-input-257-73475251b1aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_100_sadie_airr_only["v_call_top"] = (
<ipython-input-257-73475251b1aa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_100_sadie_airr_only["d_call_top"] = (
<ipython-input-257-73475251b1aa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [268]:
ref["v_identity"] // 1

82.0

In [274]:
ref["np1"]
target["np1"]

'CCTGGACCC'

In [275]:
ref["np1"]

'CCTGGACC'

In [335]:
top_100_imgt_airr_onl.

Index(['0_G_d22_H_igblastn_anarci_Heavy_IGHA_219', '1_705010661_igblastn_anarci_Heavy_IGHE_1125', '2_Subject-53_igblastn_anarci_Heavy_IGHD_22592', '3_Donor03+IGH-Clonotypes_igblastn_anarci_Heavy_Bul', '4_704010461_igblastn_anarci_Heavy_IGHE_8', '5_E_d23_H_igblastn_anarci_Heavy_IGHD_3268', '6_BCP9-GC_igblastn_anarci_Heavy_IGHD_269', '7_BCP9-Naive_igblastn_anarci_IGHD_177', '8_BCP3-GC_igblastn_anarci_Heavy_IGHD_2256', '9_SRR12081566_igblastn_anarci_Heavy_Bulk_22358', '10_703010269_igblastn_anarci_Heavy_IGHD_33102', '11_707010476_igblastn_anarci_Heavy_IGHD_5257', '12_703010632_igblastn_anarci_Heavy_IGHG_27951', '13_Donor01+IGH-Clonotypes_igblastn_anarci_Heavy_Bu', '14_705010661_igblastn_anarci_Heavy_IGHA_18220', '15_Subject-50_igblastn_anarci_Heavy_IGHD_68196', '16_BCP3-MBC_igblastn_anarci_IGHG_8632', '17_Subject-19_igblastn_anarci_Heavy_IGHG_32336', '18_Subject-71_igblastn_anarci_Heavy_IGHD_1288', '19_BCP2-GC_igblastn_anarci_IGHG_6595', '20_SRR12081539_igblastn_anarci_Heavy_Bulk_32454', 

In [356]:
target_all = top_100_sadie_airr_only.reset_index()
reference_all = top_100_imgt_airr_only.reset_index()
for index in reference_all.index:
    ref = reference_all.loc[index]
    target = target_all.loc[index]
    if (
        ref["v_call_top"] == target["v_call_top"]
        and ref["j_call_top"] == target["j_call_top"]
    ):
        check_these = ["fwr1", "cdr1", "fwr2", "cdr2", "fwr3", "cdr3"]
        if not (ref[check_these].fillna("") == target[check_these].fillna("")).all():
            print("here")
            break
    else:
        print(
            ref["v_call_top"],
            target["v_call_top"],
            ref["j_call_top"],
            target["j_call_top"],
        )
    # diff_indexes = ref[~(ref == target)].index


#     for different_index in diff_indexes:
#         ref_ = ref[different_index]
#         target_ = target[different_index]
#         if isinstance(ref_, float):
#             if round(ref_, 1) == round(target_, 1):
#                 continue
#         print(f"{different_index}\nIMGT: {ref_}\nSADIE: {target_}\n")
#     break

IGHV3-30-3*01 IGHV3-30-3*01 nan IGHJ2*01
IGHV4-38-2*01 IGHV4-38-2*02 IGHJ5*02 IGHJ5*02
IGHV4-34*08 IGHV4-34*01 IGHJ4*02 IGHJ4*02
IGHV1-2*02 IGHV1-2*02 IGHJ5*01 IGHJ4*03
IGHV5-51*01 IGHV5-51*01 IGHJ5*01 IGHJ4*02
IGHV4-39*01 IGHV4-39*01 IGHJ4*02 IGHJ5*02
IGHV3-30*03 IGHV3-30*18 IGHJ6*02 IGHJ6*02
IGHV1-69-2*01 IGHV1-69-2*01 IGHJ4*03 IGHJ4*01
IGHV4-59*09 IGHV4-61*02 IGHJ4*02 IGHJ4*02
IGHV1-69*07 IGHV1-69*05 nan IGHJ6*01
IGHV4-39*07 IGHV4-59*01 IGHJ4*02 IGHJ4*02
IGHV3-30*03 IGHV3-30*18 IGHJ6*02 IGHJ6*02
IGHV4-38-2*02 IGHV4-30-4*05 IGHJ6*01 IGHJ6*01
here


In [358]:
ref[check_these] == target[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2    False
fwr3     True
cdr3     True
Name: 99, dtype: bool

In [362]:
ref["cdr2_aa"]

'ISGSRTYI'

In [363]:
target["cdr2_aa"]

'ISGSFRTYI'

In [361]:
ref["sequence"]

'TACCTTGACTTAACTCTTGGGGGTCTGGGGGAGGCCTGGTCAAGCCTGGGGGGTCCCTGAGACTCTCATGTGCAGCCTCTGGATTCACCTTCAGTAGTTTTACCATGAATTGGGTCCGCCAGGCTCCAGGGAAGGGACTGGAGTGGGTCTCATCCATTAGCGGCAGTTTCAGAACTTACATATATTATGCCGACTCAGTGAAGGGCCGATTCACCGTCTCCAGAGACAACGCCAAGGAATTGGTGTTTCTGCAGATGGACAACCTGAGAGTCGAAGACACAGGTGTATATTATTGTGCGAGAGACCTAAATACGGTGACTACCCCAGAATACTTCCAACACTGGGGCCTGGGCACCCCGGTCTCCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTG'

In [355]:
target["cdr2"]

'ATCAACCCTAGTGGTGATAACACA'

In [344]:
ref["fwr4"]

'TGGGGCCAGGGAACC'

In [345]:

target["fwr4"]

'TGGGGCCAGGGAACCCT'

In [342]:
ref[check_these] == target[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2     True
fwr3     True
cdr3     True
fwr4    False
Name: 3, dtype: bool

In [328]:
len(ref["fwr4"])

33

In [320]:
ref["v_call_top"]

'IGHV4-31*02'

In [321]:
target["v_call_top"]

'IGHV4-31*02'

In [303]:
ref["j_call_top"]

'IGHJ4*02'

In [304]:
target["j_call_top"]

'IGHJ4*02'

In [330]:
len(target["fwr4"])

33

In [331]:
len(ref["fwr4"])

33

In [ ]:
TGGGGCCAGGGAACCCTGGTCACCGTCGCCTCAG
TGGGGCCAGGGAACCCTGGTCACCGTCGCCTCA

In [312]:
from Bio.Seq import Seq

Seq("tggggccagggaaccctggtcaccgtcgcctca").translate()

Seq('WGQGTLVTVAS')

In [308]:
ref["sequence"]

'GGCCTCAGTGAAGGTCTCCTGCAAGGCATCTGGATACACCTTCACCAACCACTACATCCACTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGAATAATCAACCCTAGTGGTGGTAGGACAAGTCACGTACAGGAGTTCCAGGGCAGAGTCACCATGACCAGGGACACGTCCACGAGTACTGTCTACATGGAGTTGAGTAGCCTGAGATCTGAGGACACGGCCGTGTACTTTTGTGCGAGACAACAGTGTAGTGCTTCCAACTGCTATGAGGAGAGTTTTGACCACTGGGGCCAGGGAACCCTGGTCACCGTCGCCTCAGCCTCCACACAGAGCCCATCCGTCTTC'

In [298]:
target[check_these] == ref[check_these]

fwr1     True
cdr1     True
fwr2     True
cdr2     True
fwr3     True
cdr3     True
fwr4    False
Name: 1_705010661_igblastn_anarci_Heavy_IGHE_1125, dtype: bool

'ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCTGGATCCGCCAGCACCCAGGGAAGGGCCTGGAGTGGATTGGGTACATCTATTACAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTTACCATATCAGTAGACACGTCTAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACTGCCGCGGACACGGCCGTGTATTACTGTGCGANNNNNNNNNAACTCCNNNNTGATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCA'

In [193]:
spread.df_to_sheet(
    fillna(top_100_sadie_airr_only),
    sheet="sadie_airr_only",
    freeze_headers=True,
    freeze_index=True,
    replace=True,
)

<ipython-input-192-d3342252a2a0>:16: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  df.dtypes.apply(lambda x: x in ["float64", "Int16"])


In [213]:
spread.df_to_sheet(
    fillna(top_100_imgt_airr_only),
    sheet="imgt_airr_only",
    freeze_headers=True,
    freeze_index=True,
    replace=True,
)

<ipython-input-210-72c7d47bbfa2>:16: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  df.dtypes.apply(lambda x: x in ["float64", "Int16", "Int64"])


germline_alignment         ACTGTCTCTGGTGGCTCCATCAGCAGTGGTGGTTACTACTGGAGCT...
germline_alignment_aa      TVSGGSISSGGYYWSWIRQHPGKGLEWIGYIYYSGSTYYNPSLKSR...
v_alignment_end                                                       316.00
v_germline_start                                                        1.00
v_germline_end                                                        316.00
v_identity                                                             82.22
d_alignment_start                                                     325.00
d_alignment_end                                                       334.00
d_germline_alignment                                              CAACTGGAAC
d_germline_alignment_aa                                                  QLE
d_germline_start                                                        5.00
d_germline_end                                                         14.00
d_identity                                                              0.00

## Try and find OAS similariities

I'm suspect of the entire airr file since it doesn't have fwr4

In [ ]:
df_bz2 = pd.read_csv(
    "https://sadie.s3.us-east-2.amazonaws.com/integration/OAS_sample_subsample.bz2",
    index_col=0,
).reset_index()

In [ ]:
airr_dataframe = airr_api.run_dataframe(df_bz2, "index", "sequence")

In [ ]:
ref = df_bz2[df_bz2.columns.intersection(airr_dataframe.table.columns)].drop(
    "sequence_id", axis=1
)
target = airr_dataframe.table[
    airr_dataframe.table.columns.intersection(df_bz2.columns)
].drop("sequence_id", axis=1)

In [ ]:
ref.re

In [ ]:
for index in ref.index:
    sub_ref = ref.loc[index]
    sub_target = target.loc[index]
    aggree = sub_ref[sub_ref == sub_target].index
    disagree = sub_ref[sub_ref != sub_target].index
    if disagree.empty:
        print('yay')
    for dis_index in disagree:
        print(f"OAR-{dis_index}:{sub_ref[dis_index]}")
        print(f"Sadie-{dis_index}:{sub_target[dis_index]}\n")

    break

In [ ]:
disagree

In [ ]:
from pandas.testing import assert_series_equal

# Get IMGT airr

In [ ]:
imgt_airr = "../../tests/integration/airr/fixtures/OAS_airr_from_imgtvquest.tsv"
imgt_df = pd.read_csv(imgt_airr, delimiter="\t")

In [ ]:
spread.df_to_sheet(imgt_df, sheet="imgt_airr")